<a href="https://colab.research.google.com/github/christianwarmuth/openhpi-kipraxis/blob/main/Woche%202/2_7_Ergebnis_und_Auswertung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installieren aller Pakete

In [ ]:
# Hier die Kaggle Credentials einfügen (ohne Anführungszeichen)

%env KAGGLE_USERNAME=openhpi
%env KAGGLE_KEY=das_ist_der_key

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel

from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
from keras.models import Model

from sklearn.decomposition import NMF

import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = [5, 3]
plt.rcParams['figure.dpi'] = 150 

# Content-based Filtering

In [ ]:
!pip3 install kaggle
!kaggle datasets download -d rounakbanik/the-movies-dataset

In [ ]:
import zipfile
with zipfile.ZipFile("the-movies-dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("")

In [ ]:
df_film_metadata = pd.read_csv("movies_metadata.csv", low_memory=False)

In [ ]:
df_film_metadata = df_film_metadata[df_film_metadata['overview'].notna()]
df_film_metadata = df_film_metadata.reset_index()

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_film_metadata['overview'])

title_to_index = pd.Series(df_film_metadata.index, index=df_film_metadata['title'])

movie_matrix=csr_matrix(tfidf_matrix)
model_knn= NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=10, n_jobs=-1)

def recommend_films_by_title_knn(title, data, model, n_neighbors):
    model.fit(data)
    movie_idx = title_to_index[title]
    sim_scores, movie_indices = model.kneighbors(data[movie_idx], n_neighbors=n_neighbors+1)
    sim_scores = sim_scores.squeeze().tolist()
    recommendation_list = []
    for idx, movie_idx in enumerate(movie_indices.squeeze().tolist()):
        recommendation_list.append({'Title':df_film_metadata['title'][movie_idx],'Similarity':sim_scores[idx]})
    return pd.DataFrame(recommendation_list).sort_values(by=['Similarity'], ascending=False).reset_index(drop=True)[:-1]

# Collaborative Filtering

In [ ]:
df_film_ratings = pd.read_csv("ratings_small.csv", low_memory=False)

In [ ]:
df_film_ratings = df_film_ratings.drop("timestamp", axis=1) # Entfernen des Spalte Timestamp
df_film_ratings = df_film_ratings.sample(frac=1).reset_index(drop=True) # Durchmischen der Bewertungen

split_factor = 0.2
n = int(split_factor*len(df_film_ratings))

df_train = df_film_ratings[:-n]
df_test = df_film_ratings[-n:]

# Credits: https://www.kaggle.com/morrisb/how-to-recommend-anything-deep-recommender

def mf_model(df, train, test,  emb_size=20):
    # Create user- & movie-id mapping
    user_id_mapping = {id:i for i, id in enumerate(df['userId'].unique())}
    movie_id_mapping = {id:i for i, id in enumerate(df['movieId'].unique())}
    
    
    # Create correctly mapped train- & testset
    train_user_data = train['userId'].map(user_id_mapping)
    train_movie_data = train['movieId'].map(movie_id_mapping)
    
    test_user_data = test['userId'].map(user_id_mapping)
    test_movie_data = test['movieId'].map(movie_id_mapping)
    
    
    # Get input variable-sizes
    users = len(user_id_mapping)
    movies = len(movie_id_mapping)
    embedding_size = emb_size
    
    
    ##### Create model
    # Set input layers
    user_id_input = Input(shape=[1], name='user')
    movie_id_input = Input(shape=[1], name='movie')
    
    # Create embedding layers for users and movies
    user_embedding = Embedding(output_dim=embedding_size, 
                               input_dim=users,
                               input_length=1, 
                               name='user_embedding')(user_id_input)
    movie_embedding = Embedding(output_dim=embedding_size, 
                                input_dim=movies,
                                input_length=1, 
                                name='item_embedding')(movie_id_input)
    
    # Reshape the embedding layers
    user_vector = Reshape([embedding_size])(user_embedding)
    movie_vector = Reshape([embedding_size])(movie_embedding)
    
    # Compute dot-product of reshaped embedding layers as prediction
    y = Dot(1, normalize=False)([user_vector, movie_vector])
    
    # Setup model
    model = Model(inputs=[user_id_input, movie_id_input], outputs=y)
    model.compile(loss='mse', optimizer='adam')
    return train_user_data, train_movie_data, test_user_data, test_movie_data, model

train_user_data, train_movie_data, test_user_data, test_movie_data, model = mf_model(df_film_ratings, df_train, df_test)

history = model.fit([train_user_data, train_movie_data],
          df_train['rating'],
          batch_size=256, 
          validation_split=0.1,
          epochs=10,
          shuffle=True)

# 2.7 Ergebnis und Auswertung

<img width=70% src="https://raw.githubusercontent.com/christianwarmuth/openhpi-kipraxis/main/images/jakob-owens-CiUR8zISX60-unsplash%20(2).jpg">


In diesem Notebook wollen wir einmal das erstellte Content-based Recommendation System und das Collaborative Filtering Recommendation System auswerten. Auch wenn wir in beiden Fällen jeweils Vorschläge für Filme geben, werden wir sehen, dass die Möglichkeiten zur Auswertung sich deutlich unterscheiden werden. 

## Content-based Recommendation - Ergebnis und Auswertung

Die verwendete Content-based Recommendation ist eine unsupervised Machine Learning Technik. Wir haben hierbei keinerlei Labels, ob ein Film einem anderen inhaltlich ähnlich ist. Die inhaltiche Ähnlichkeit ist daher sehr schwer zu überprüfen. Bei unsupervised Techniken ist es meist nur für Domain-Experten möglich, die "Güte" der Ergebnisse zu bewerten.

Wir können uns erneut einmal die Vorschläge zu dem Film "Golden Eye" ausgeben lassen. Golden Eye ist ein Film aus der James Bond Reihe. Wir sehen hier, dass viele der Vorschläge tatsächlich andere James Bond Filme sind (z.B: Casion Royale, Never Say Never Again, You Only Live Twice, Octopussy, Live and Let Die, Licence to Kill).

In [ ]:
recommend_films_by_title_knn("GoldenEye", movie_matrix, model_knn, n_neighbors=10)

Nun sehen wir uns einmal die Kurzbeschreibungen der Filme in der Liste an, die keine James Bond Filme sind. Das gibt uns vielleicht einen Hinweis, warum die Filme inhaltlich ähnlich zu James Bond Filmen gesehen werden.  

In [ ]:
print("The Way of the Dragon: " + df_film_metadata["overview"][title_to_index["The Way of the Dragon"]] + "\n")
print("Johnny Stool Pigeon: " + df_film_metadata["overview"][title_to_index["Johnny Stool Pigeon"]] + "\n")
print("Doctor X: " + df_film_metadata["overview"][title_to_index["Doctor X"]] + "\n")
print("Dream Work: " + df_film_metadata["overview"][title_to_index["Dream Work"]] + "\n")

Es wäre allerdings sehr falsch, unsere Bewertung unserer Ergebnisse nur anhand eines Vorschlages zu testen. Daher geben wir uns weitere Vorschläge zu weiteren bekannten Filmen aus. Hier sieht man jedoch ein besseres Ergebnis, da alle Filme zumindest einmal den Titel Batman tragen. Interessant ist hier zu sehen, dass teilweise Comic-Filme, Dokumentationen über die Personen hinter Batman und alte und neue Batman Filme vertreten sind.  

In [ ]:
recommend_films_by_title_knn("Batman Begins", movie_matrix, model_knn, n_neighbors=10)

Als letztes Experiment sehen wir uns einen weiteren Film-Klassiker an:

In [ ]:
recommend_films_by_title_knn("Star Wars: Episode II - Attack of the Clones", movie_matrix, model_knn, n_neighbors=10)

Abschließend sollten wir erneut festhalten: Die "Güte" von unsupervised Modellen ist ohne Domain-Wissen sehr schwer bewertbar. Wenn man einen unsupervised Ansatz plant einzusetzen, so sollte man sich stets über die anschließende Evaluation Gedanken machen. 

## Collaborative Filtering - Ergebnis und Auswertung

Im Gegensatz zum vorgestellten Content-based Ansatz, haben wir für den Collaborative Filtering Ansatz sogenannte Labels. Wir wir bereits in der Session 2.6 erwähnt haben, teilen wir hier auch den Datensatz in "Test" und "Training". In diesem Fall sind die Ergebnisse weniger einfach "interpretierbar", doch haben wir in diesem Fall eine "Ground Truth" - also die Bewertung die wirklich gegeben wurde. Als Metrik zur Bewertung der "Modell-Güte" betrachten wir den Root Mean Squared Error, der die Abweichung des vorhergesagten Wertes vom tatsächlichen Wert beschreibt. 

Um das Modell-Training etwas genauer zu verstehen, können wir uns einmal die Kurve der Loss-Funktion über das Training hinweg ansehen. Der Loss beschreibt den "Fehler" (Abweichung wahrer Wert vs. Vorhersage), der während des Trainings (in diesem Falle) entweder auf den Trainings oder den Validierungsdaten existiert. Man sollte das Training erst beenden, wenn sich dieser Wert nur noch sehr minimal verändert.

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')

In [ ]:
# Test model
y_pred = model.predict([test_user_data, test_movie_data])
y_true = df_test['rating'].values

#  Berechne Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_true))
print('RMSE: '+ str(rmse))

In [ ]:
print("Wahrer Wert: " + str(y_true[3]) + "\n") 
print("Vorhersage:  " + str(y_pred[3][0]))

Auch wenn die obere Art der Auswertung eingängier zu sein scheint, ist die Auswertung des Collaborative Filtering Ansatzes natürlich deutlich objektiver, da wir die wahren Lables/Bewertungen haben und uns damit vergleichen können. 